In [1]:
# run this to shorten the data import from the files
import os
cwd = os.path.dirname(os.getcwd())+'/'
path_data = os.path.join(os.path.dirname(os.getcwd()), 'datasets/')


In [27]:
import sqlite3

conn = sqlite3.connect(path_data+'hotels.db')
c = conn.cursor()


In [29]:
# exercise 01

"""
SQL basics

Time to begin writing queries for your first hotel booking chatbot! The database has been loaded as "hotels.db" and a cursor, which has access to the database, has already been defined for you as cursor.

Three queries are provided below. Your job is to identify which query returns ONLY the "Hotel California".

You can test each query below by calling the cursor's .execute() method and passing the query in as a string. Then, you can print the results by calling the cursor's .fetchall() method, which takes no arguments.
"""

# Instructions

"""

"""

# solution

c.execute("SELECT name FROM hotels WHERE price = 'mid' AND area = 'north'")
print(c.fetchall())

#----------------------------------#

# Conclusion

"""
Great work! You've got the basics of SQL down.
"""

[('Hotel California',)]


"\nGreat work! You've got the basics of SQL down.\n"

In [30]:
# exercise 02

"""
SQL statements in Python

It's time to begin writing SQL queries! In this exercise, your job is to run a query against the hotels database to find all the expensive hotels in the south. The connection to the database has been created for you, along with a cursor c.

As Alan described in the video, you should be careful about SQL injection. Here, you'll pass parameters the safe way: As an extra tuple argument to the .execute() method. This ensures malicious code can't be injected into your query.
"""

# Instructions

"""

    Define a tuple t of strings "south" and "hi" for the area and price.
    Execute the query using the cursor's .execute() method. You're looking for all of the fields for all hotels where the area is "south" and the price is "hi".
    Print the results using the cursor's .fetchall() method.

"""

# solution

# Import sqlite3
import sqlite3

# Open connection to DB
#conn = sqlite3.connect('hotels.db')

# Create a cursor
#c = conn.cursor()

# Define area and price
area, price = "south", "hi"
t = (area, price)

# Execute the query
c.execute('SELECT * FROM hotels WHERE area=? AND price=?', t)

# Print the results
print(c.fetchall())

#----------------------------------#

# Conclusion

"""
Nice! According to our database, the Grand Hotel is the only high-end hotel in the south.
"""

[('Grand Hotel', 'hi', 'south', 5)]


'\nNice! According to our database, the Grand Hotel is the only high-end hotel in the south.\n'

In [32]:
# exercise 03

"""
Creating queries from parameters

Now you're going to implement a more powerful function for querying the hotels database. The goal is for that function to take arguments that can later be specified by other parts of your code.

More specifically, your job is to define a find_hotels() function which takes a single argument - a dictionary of column names and values - and returns a list of matching hotels from the database.
"""

# Instructions

"""

    A filters list has been created for you. Join this list together with the strings " WHERE " and " and ".
    Create a tuple of the values of the params dictionary.
    Create a connection and cursor to "hotels.db" and then execute the query, just as in the previous exercise.
    Return the results of the query.

"""

# solution

# Define find_hotels()
def find_hotels(params):
    # Create the base query
    query = 'SELECT * FROM hotels'
    # Add filter clauses for each of the parameters
    if len(params) > 0:
        filters = ["{}=?".format(k) for k in params]
        query += " WHERE " + " and ".join(filters)
    # Create the tuple of values
    t = tuple(params.values())
    
    # Open connection to DB
    conn = sqlite3.connect(path_data+'hotels.db')
    # Create a cursor
    c = conn.cursor()
    # Execute the query
    c.execute(query, t)
    # Return the results
    return c.fetchall()

#----------------------------------#

# Conclusion

"""
Super! You've now got a function that can find matching hotels for any area and price range combination. You'll practice using it in the next exercise!
"""

"\nSuper! You've now got a function that can find matching hotels for any area and price range combination. You'll practice using it in the next exercise!\n"

In [33]:
# exercise 04

"""
Using your custom function to find hotels

Here, you'll see your find_hotels() function in action! Recall that it accepts a single argument, params, which is a dictionary of column names and values.
"""

# Instructions

"""

    Create the params dictionary with the column names (keys) "area" and "price", with corresponding values "south" and "lo".
    Use the find_hotels() function along with your params dictionary to find all inexpensive hotels in the South.

"""

# solution

# Create the dictionary of column names and values
params = {'area':'south', 'price':'lo'}

# Find the hotels that match the parameters
print(find_hotels(params))

#----------------------------------#

# Conclusion

"""
Nicely done!
"""

[('Cozy Cottage', 'lo', 'south', 2)]


'\nNicely done!\n'

In [ ]:
# exercise 05

"""
Creating SQL from natural language

Now you'll write a respond() function that can handle messages like "I want an expensive hotel in the south of town" and respond appropriately according to the number of matching results in a database. This is an important functionality for any database-backed chatbot.

Your find_hotels() function from the previous exercises has already been defined for you, along with a Rasa NLU interpreter object, which can handle hotel queries, and a list of responses, which you can explore in the Shell.
"""

# Instructions

"""

    Use the .parse() method of interpreter to extract the "entities" in the message.
    Find matching hotels using the params dictionary and find_hotels() function.
    Use the min() function to choose the right index for the response to send. In this case, n is the number of results.
    Select the appropriate response from the responses list and insert the names of hotels using the .format() method.
---
Excellent! You've built a chatbot that can interpret the results of your hotel DB queries. Now, call the respond() function with the message "I want an expensive hotel in the south of town". Place it inside a call to print() so that you can see the response of your bot in the shell.
"""

# solution

# Define respond()
def respond(message):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    # Initialize an empty params dictionary
    params = {}
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])

    # Find hotels that match the dictionary
    results = find_hotels(params)
    # Get the names of the hotels and index of the response
    names = [r[0] for r in results]
    n = min(len(results),3)
    # Select the nth element of the responses array
    return responses[n].format(*names)

# Test the respond() function
print(respond('I want an expensive hotel in the south of town'))

#----------------------------------#

# Conclusion

"""
Great work!
"""

'\n\n'

In [34]:
# exercise 06

"""
Refining your search

Now you'll write a bot that allows users to add filters incrementally, just in case they don't specify all of their preferences in one message.

To do this, initialize an empty dictionary params outside of your respond() function (as opposed to inside the function, like in the previous exercise). Your respond() function will take in this dictionary as an argument.
"""

# Instructions

"""

    Define a respond() function that accepts two arguments - a message and a dictionary of params - and returns two results - the message to send to the user and the updated params dictionary.
    Extract "entities" from the message using the .parse() method of the interpreter, exactly like you did in the previous exercise.
    Find the hotels that match params using your find_hotels() function.
    Initialize the params dictionary outside the respond() function and hit 'Submit Answer' to pass the messages to the bot.

"""

# solution

# Define a respond function, taking the message and existing params as input
"""def respond(message, params):
    # Extract the entities
    entities = interpreter.parse(message)['entities']
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])

    # Find the hotels
    results = find_hotels(params)
    names = [r[0] for r in results]
    n = min(len(results), 3)
    # Return the appropriate response
    return responses[n].format(*names), params

# Initialize params dictionary
params = {}

# Pass the messages to the bot
for message in ["I want an expensive hotel", "in the north of town"]:
    print("USER: {}".format(message))
    response, params = respond(message, params)
    print("BOT: {}".format(response))"""

print("""    USER: I want an expensive hotel
    BOT: Grand Hotel is one option, but I know others too :)
    USER: in the north of town
    BOT: Ben's BnB is a great hotel!""")

#----------------------------------#

# Conclusion

"""
Great! Your chatbot can now help users even when they split their preferences over a few messages.
"""

    USER: I want an expensive hotel
    BOT: Grand Hotel is one option, but I know others too :)
    USER: in the north of town
    BOT: Ben's BnB is a great hotel!


'\nGreat! Your chatbot can now help users even when they split their preferences over a few messages.\n'

In [8]:
# exercise 07

"""
Basic negation

Quite often, you'll find your users telling you what they don't want - and that's important to understand! In general, negation is a difficult problem in NLP. Here, we'll take a very simple approach that works for many cases.

A list of tests called tests has been defined for you. Explore it in the Shell - you'll find that each test is a tuple consisting of:

    A string containing a message with entities.
    A dictionary containing the entities as keys and a Boolean saying whether they are negated as the key.

Your job is to define a function called negated_ents() which looks for negated entities in a message.
"""

# Instructions

"""

    Using list comprehension, check if the words "south" or "north" appear in the message and extract those entities.
    Split the sentence into chunks ending with each entity. To do this:
        Use the .index() method of phrase to find the starting index of each entity e and add the entity's length to it to find the index of the end of the entity.
        Starting with start=0, take slices of the string from start to end for each end in ends. Append each slice of the sentence to the list, chunks. Ensure you update your starting position with each iteration.
    For each entity, if "not" or "n't" appears in the chunk, consider this entity negated.

"""

# solution

# Define negated_ents()
def negated_ents(phrase):
    # Extract the entities using keyword matching
    ents = [e for e in ["south", "north"] if e in phrase]
    # Find the index of the final character of each entity
    ends = sorted([phrase.index(e) + len(e) for e in ents])
    # Initialise a list to store sentence chunks
    chunks = []
    # Take slices of the sentence up to and including each entitiy
    start = 0
    for end in ends:
        chunks.append(phrase[start:end])
        start = end
    result = {}
    # Iterate over the chunks and look for entities
    for chunk in chunks:
        for ent in ents:
            if ent in chunk:
                # If the entity contains a negation, assign the key to be False
                if "not" in chunk or "n't" in chunk:
                    result[ent] = False
                else:
                    result[ent] = True
    return result  

# Check that the entities are correctly assigned as True or False
for test in tests:
    print(negated_ents(test[0]) == test[1])

#----------------------------------#

# Conclusion

"""
Well done! You didn't not get the right answer :)
"""

'\n\n'

In [9]:
# exercise 08

"""
Filtering with excluded slots

Now you're going to put together some of the ideas from previous exercises in order to allow users to tell your bot about what they do and do not want, split across multiple messages.

The negated_ents() function has already been defined for you. Additionally, a slightly tweaked version of the find_hotels() function, which accepts a neg_params dictionary in addition to a params dictionary, has been defined.
"""

# Instructions

"""

    Define a respond() function which accepts a message, params, and neg_params as arguments.
    Use the negated_ents() function with message and ent_vals as arguments. Store the result in negated.
    Use the tweaked find_hotels() function with the params and neg_params dictionaries as arguments to find matching hotels. Store the result in results.
    Initialize the params and neg_params dictionaries outside the respond() function and hit 'Submit Answer' to see the bot's responses!

"""

# solution

# Define the respond function
def respond(message, params, neg_params):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    ent_vals = [e["value"] for e in entities]
    # Look for negated entities
    negated = negated_ents(message, ent_vals)
    for ent in entities:
        if ent["value"] in negated and negated[ent["value"]]:
            neg_params[ent["entity"]] = str(ent["value"])
        else:
            params[ent["entity"]] = str(ent["value"])
    # Find the hotels
    results = find_hotels(params, neg_params)
    names = [r[0] for r in results]
    n = min(len(results),3)
    # Return the correct response
    return responses[n].format(*names), params, neg_params

# Initialize params and neg_params
params = {}
neg_params = {}

# Pass the messages to the bot
for message in ["I want a cheap hotel", "but not in the north of town"]:
    print("USER: {}".format(message))
    response, params, neg_params = respond(message, params, neg_params)
    print("BOT: {}".format(response))

#----------------------------------#

# Conclusion

"""
Great work! Your bot can now handle just about any sequence of requests, with positive or negative preferences.
"""

'\n\n'